In [68]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import json
import csv
import requests
import lxml
import re

In [15]:
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)


## 下載普通适水平測試字表


In [ ]:
def word_one_url_generator():
    template = "http://www.pthxx.com/b_audio/03_zici_1/%02d.html"
    max_pg =66
    for i in range(1, max_pg+1):
        yield template % i
    
def word_two_url_generator():
    template = "http://www.pthxx.com/b_audio/04_zici_2/%03d.html"
    max_pg =105
    for i in range(1, max_pg+1):
        yield template % i

In [ ]:
def extract_word_list(url):
    result = requests.get(url)
    c= result.content
    soup = BeautifulSoup((c), "lxml")

    soup.prettify()

    summary = soup.find_all('div',attrs = {'id':re.compile(r'main[0-9]')})
    tables = []
    for div in summary:
        tables_ = div.find_all('table')
        tables.extend(tables_)

    data =[]

    for table in tables:
        #print('debug', table)
        rows = table.findAll('tr')
        for row in table.findAll('tr')[0:]:
            for word,pinyin in pairwise(row.findAll('td')[1:]):
                data.append((word.text.replace(' ',''), pinyin.text.replace(' ','')))

    return data

In [ ]:
word_one_gen = word_one_url_generator()

all_data = []
for url in word_one_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)
    
print(all_data)

In [ ]:
with open('./word_one.txt', 'w', newline='', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for pair in all_data:
        wr.writerow(pair)

In [ ]:
word_two_gen = word_two_url_generator()

all_data = []
for url in word_two_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)
    
print(all_data)

In [ ]:
page_data = extract_word_list(url)
print('debug', url, 'vocab: ', len(page_data))
all_data.extend(page_data)

In [ ]:
for url in word_two_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)

In [ ]:
with open('./word_two.txt', 'w', newline='', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for pair in all_data:
        wr.writerow(pair)

## 讀取字表與文章

In [16]:
import pandas as pd

# Read the CSV into a pandas data frame (df)
#   With a df you can do many things
#   most important: visualize data with Seaborn
df = pd.read_csv('./word_one.txt', delimiter=',')

# Or export it in many ways, e.g. a list of tuples
word_1_list = [tuple(x) for x in df.values]

In [17]:
df = pd.read_csv('./word_two.txt', delimiter=',')

# Or export it in many ways, e.g. a list of tuples
word_2_list = [tuple(x) for x in df.values]

In [18]:
passages = []

for i in range(1,61):
    with open('./passage/%03d.txt' % i, encoding='UTF-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    ann_list = []
    for word,pinyin in pairwise(content):
        ann_list.append((word, pinyin))
    passages.append(ann_list)

## 提取表1,2字的拼音

In [19]:
a_list = ['a', 'ā', 'á', 'ǎ', 'à']
o_list = ['o','ō', 'ó', 'ǒ', 'ò']
e_list = ['e', 'ē', 'é', 'ě', 'è']
i_list = ['i', 'ī', 'í', 'ǐ', 'ì']
u_list = ['u', 'ū', 'ú', 'ǔ', 'ù']
yu_list = ['ü', 'ǖ', 'ǘ', 'ǚ', 'ǜ']

consonant = ['b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'g', 'k', 'h', 'j', 'q', 'x', 'zh', 'ch', 'sh', 'r', 'z', 'c', 's', 'y', 'w']
sep = '’'
vowel = ['a', 'o', 'e', 'ai', 'ei', 'ao', 'ou', 'an', 'en', 'ang', 'eng', 'ong', 'i', 'ia', 'ie', 'iao', 'iou',
         'ian', 'in', 'iang', 'ing', 'iong', 'u', 'ua', 'uo', 'uai', 'uei', 'uan', 'uen', 'uang', 'ueng', 
         'ü', 'üe', 'üan', 'ün',
         'ue',
         'iu', 'ui', 'un' 
        ]

In [104]:
def remove_tone(alpha):
    if alpha in a_list:
        return 'a'
    elif alpha in o_list:
        return 'o'
    elif alpha in e_list:
        return 'e'
    elif alpha in i_list:
        return 'i'
    elif alpha in u_list:
        return 'u'
    elif alpha in yu_list:
        return 'ü'
    else:
        return alpha

def word_sep(word, pinyin):
    pinyin_t = ''.join([remove_tone(p) for p in pinyin])
    pinyin_len = len(pinyin)
    #print('debug', pinyin)
    result = []
    check_cnt= 0
    for w in word:
        const_p = ""
        vowel_p = ""
        
        if pinyin_t.startswith(sep) or pinyin_t.startswith("•") or pinyin_t.startswith(" "):
            pinyin_t = pinyin_t[1:]
            pinyin = pinyin[1:]
            check_cnt = check_cnt + 1
            
        for c in consonant:
            if(pinyin_t.startswith(c)):
                if(len(const_p) < len(c)):
                    const_p = c
                    
        const_p = pinyin[:len(const_p)]
        pinyin_t = pinyin_t[len(const_p):]
        pinyin = pinyin[len(const_p):]
        
        for v in vowel:
            if(pinyin_t.startswith(v)):
                #print('debugh xx', v, pinyin_t)
                if(len(vowel_p) < len(v)):
                    vowel_p = v
                    
        if len(const_p) == 0 and vowel_p == "e":
            if(pinyin_t.startswith("er")):
                vowel_p = pinyin[0:2]
        
        vowel_p = pinyin[:len(vowel_p)]
        pinyin_t = pinyin_t[len(vowel_p):]
        pinyin = pinyin[len(vowel_p):]
        #print("debug V", vowel_p, pinyin)
        result.append((w, const_p+vowel_p))
        check_cnt = check_cnt + len(const_p) + len(vowel_p)
    
    if check_cnt != pinyin_len:
        raise Exception("not all pinyin is used. %d %d" % (check_cnt, pinyin_len) + str(result) )
        
    return result
    
def character_tone(pinyin):
    '''
    pass back 1, 2, 3, 4 or 0
    '''
    for c in pinyin:
        a_idx = a_list.index(c) if c in a_list else -1
        o_idx = o_list.index(c) if c in o_list else -1
        e_idx = e_list.index(c) if c in e_list else -1
        i_idx = i_list.index(c) if c in i_list else -1
        u_idx = u_list.index(c) if c in u_list else -1
        yu_idx = yu_list.index(c) if c in yu_list else -1
        
        if(a_idx > 0):
            return a_idx
        elif(o_idx > 0):
            return o_idx
        elif(e_idx > 0):
            return e_idx
        elif(i_idx > 0):
            return i_idx
        elif(u_idx > 0):
            return u_idx
        elif(yu_idx > 0):
            return yu_idx
        
    return 0

def create_character_pinyin_dict(word_list):
    '''
    dictionary lookup by word, then each sound have a list of words.
    '''
    char_dict = {}
    for word in word_list:
        for char, pinyin in word:
            if not char in char_dict:
                pinyin_dict = {}
                pinyin_dict[pinyin] = [word]
                char_dict[char] = pinyin_dict
            else:
                pinyin_dict = char_dict[char]
                if pinyin in pinyin_dict:
                    pinyin_dict[pinyin].append(word)
                else:
                    pinyin_dict[pinyin] = [word]
                    
    return char_dict

def create_pinyin_character_dict(word_list):
    '''
    dictionary lookup by pinyin, then each character have a list of words.
    '''
    pinyin_dict = {}
    for word in word_list:
        for char, pinyin in word:
            if not pinyin in pinyin_dict:
                char_dict = {}
                char_dict[char] = [word]
                pinyin_dict[pinyin] = char_dict
            else:
                char_dict = pinyin_dict[pinyin]
                if char in char_dict:
                    char_dict[char].append(word)
                else:
                    char_dict[char] = [word]
                    
    return pinyin_dict

In [21]:
word_1_list_sep = []

for word,pinyin in word_1_list:
    try:
        test=word_sep(word,pinyin) 
        
        if len(test) == 0:
            print("Error in processing: ", word,pinyin)
        else:
            word_1_list_sep.append(test)
    except Exception as e:
        print("Error in processing: ", word,pinyin, e)

In [22]:
word_2_list_sep = []

for word,pinyin in word_2_list:
    try:
        test=word_sep(word,pinyin) 
        if len(test) == 0:
            print("Error in processing: ", word,pinyin)
        else:
            word_2_list_sep.append(test)
    except Exception as e:
        print("Error in processing: ", word,pinyin, e)

In [23]:
total_sep_list = word_1_list_sep + word_2_list_sep
print(total_sep_list[0:10])

[[('阿', 'ā'), ('姨', 'yí')], [('挨', 'āi')], [('挨', 'ái')], [('矮', 'ǎi')], [('愛', 'ài')], [('愛', 'ài'), ('國', 'guó')], [('愛', 'ài'), ('好', 'hào')], [('愛', 'ài'), ('護', 'hù')], [('愛', 'ài'), ('情', 'qíng')], [('愛', 'ài'), ('人', 'ren')]]


In [31]:
char_dict = create_character_pinyin_dict(total_sep_list)
pinyin_dict = create_pinyin_character_dict(total_sep_list)

In [134]:
char_dict['黏']

{'nián': [[('黏', 'nián')]]}

### 多音字

In [34]:
for key, value in char_dict.items():
    pinyin_dict_t = char_dict[key]
    if len(pinyin_dict_t.keys()) > 1:
        print(key + " : " + str(list(pinyin_dict_t.keys())))

阿 : ['ā', 'ē']
挨 : ['āi', 'ái']
好 : ['hào', 'hǎo']
情 : ['qíng', 'qing']
人 : ['ren', 'rén']
靜 : ['jìng', 'jing']
心 : ['xīn', 'xin']
置 : ['zhì', 'zhi']
中 : ['zhōng', 'zhòng']
會 : ['huì', 'kuài']
巴 : ['bā', 'ba']
扒 : ['bā', 'pá']
把 : ['bǎ', 'bà']
兒 : ['r', 'ér', 'er']
白 : ['bái', 'bai']
色 : ['sè', 'shǎi']
動 : ['dòng', 'dong']
家 : ['jiā', 'jia']
公 : ['gōng', 'gong']
事 : ['shì', 'shi']
扮 : ['bàn', 'ban']
樣 : ['yàng', 'yang']
乾 : ['gàn', 'gān', 'qián']
子 : ['zǐ', 'zi']
炮 : ['bāo', 'pào', 'páo']
薄 : ['báo', 'bó', 'bò']
和 : ['hé', 'hè', 'hú', 'huó', 'huò', 'huo']
石 : ['shí', 'dàn']
道 : ['dào', 'dao']
露 : ['lù', 'lòu']
發 : ['fā', 'fà', 'fa']
炸 : ['zhà', 'zhá']
背 : ['bēi', 'bèi']
方 : ['fāng', 'fang']
奔 : ['bēn', 'bèn']
性 : ['xìng', 'xing']
重 : ['zhòng', 'zhóng', 'chóng']
要 : ['yào', 'yāo']
更 : ['gēng', 'gèng']
量 : ['liàng', 'liáng', 'liang']
便 : ['biàn', 'pián']
別 : ['bié', 'biè']
達 : ['dá', 'da']
餅 : ['bǐng', 'bing']
屏 : ['bǐng', 'píng']
長 : ['cháng', 'zhǎng']
剝 : ['bō', 'bāo']
削 : ['xuē', 'xiā

### 同音字查詢

In [233]:
def query_same_sound(char):
    p_dict = char_dict[char]
    for key in p_dict.keys():
        print(key + " : " + str(list(pinyin_dict[key].keys())))
        
query_same_sound("職")

zhí : ['值', '直', '職', '殖', '執', '植', '侄']


### 拼音查詢

In [ ]:
a_list = ['a', 'ā', 'á', 'ǎ', 'à']
o_list = ['o','ō', 'ó', 'ǒ', 'ò']
e_list = ['e', 'ē', 'é', 'ě', 'è']
i_list = ['i', 'ī', 'í', 'ǐ', 'ì']
u_list = ['u', 'ū', 'ú', 'ǔ', 'ù']
yu_list = ['ü', 'ǖ', 'ǘ', 'ǚ', 'ǜ']

In [236]:
pinyin_dict["rě"]

{'惹': [[('惹', 'rě')]]}

## 文章朗讀難點

In [139]:
from IPython.display import Markdown, display

def printmd(string, color=None): 
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) 
    display(Markdown(colorstr)) 
    
def formatmd(string, color=None): 
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) 
    return colorstr

In [37]:
import thulac

thu1 = thulac.thulac()  #默认模式
text = thu1.cut("我爱北京天安门", text=True)  #进行一句话分词
print(text)

Model loaded succeed
我_r 爱_v 北京_ns 天安门_ns


### 文章分詞

In [128]:
passage_no = 3

passage_str = ""
for py, ch in passages[passage_no-1]:
    if not ch == "//": 
        passage_str = passage_str + ch
    
print(passage_str) 
text = thu1.cut(passage_str, text=True)  #进行一句话分词
print(text)

我常常遺憾我家門前那塊醜石，它黑黝黝地臥在那裏，牛似的模樣；誰也不知道是什麼時候留在這裏的，誰也不去理會它。只是麥收時節，門前攤了麥子，奶奶總是說：這塊醜石，多佔地面呀，抽空把它搬走吧。它不像漢白玉那樣的細膩，可以刻字雕花，也不像大青石那樣的光滑，可以供來浣紗捶布。它靜靜地臥在那裏，院邊的槐陰沒有庇覆它，花兒也不再在它身邊生長。荒草便繁衍出來，枝蔓上下，慢慢地，它竟銹上了綠苔、黑斑。我們這些做孩子的，也討厭起它來，曾合夥要搬走它，但力氣又不足；雖時時咒罵它，嫌棄它，也無可奈何，只好任它留在那裏了。終有一日，村子裏來了一個天文學家。他在我家門前路過，突然發現了這塊石頭，眼光立即就拉直了。他再沒有離開，就住了下來；以後又來了好些人，都說這是一塊隕石，從天上落下來已經有二三百年了，是一件了不起的東西。不久便來了車，小心翼翼地將它運走了。這使我們都很驚奇，這又怪又醜的石頭，原來是天上的啊！它補過天，在天上發過熱、閃過光，我們的先祖或許仰望過它，它給了他們光明、向往、憧憬；而它落下來了，在污土裏，荒草裏，一躺就是幾百年了！我感到自己的無知，也感到了醜石的偉大，我甚至怨恨它這麼多年竟會默默地忍受着這一切！而我又立即深深地感到它那種不屈於誤解、寂寞的生存的偉大。
我_r 常常_d 遺憾_v 我_r 家_n 門_g 前_f 那_r 塊_q 醜石_n ，_w 它_r 黑黝黝_a 地_u 臥_v 在_p 那_r 裏_q ，_w 牛_n 似_g 的_u 模樣_n ；_w 誰_g 也_d 不_d 知道_v 是_v 什麼_r 時候_v 留_v 在_p 這裏_a 的_u ，_w 誰_g 也_d 不_d 去_v 理會_v 它_r 。_w 只是_d 麥收_v 時節_n ，_w 門_g 前_f 攤_v 了_u 麥子_n ，_w 奶奶_n 總_g 是_v 說_g ：_w 這塊_v 醜石_n ，_w 多_a 佔_q 地面_n 呀_u ，_w 抽空_v 把_p 它_r 搬走_v 吧_u 。_w 它_r 不_d 像_v 漢白玉_n 那_r 樣_q 的_u 細膩_n ，_w 可以_v 刻字_v 雕花_n ，_w 也_d 不_d 像_p 大青石_n 那樣_r 的_u 光滑_a ，_w 可以_v 供_v 來_g 浣_g 紗_g 捶布_v 。_w 它_r 靜靜_a 地_u 臥_v 在_p 那_r 裏_q ，_w 院

In [129]:
import jieba

seg_list = jieba.cut(passage_str, cut_all=False)
seg_list = list(seg_list)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

Full Mode: 我/ 常常/ 遺憾/ 我家/ 門前/ 那塊/ 醜石/ ，/ 它/ 黑黝黝/ 地臥/ 在/ 那/ 裏/ ，/ 牛/ 似的/ 模樣/ ；/ 誰/ 也/ 不/ 知道/ 是/ 什麼/ 時候/ 留在/ 這/ 裏/ 的/ ，/ 誰/ 也/ 不/ 去/ 理會/ 它/ 。/ 只是/ 麥收/ 時節/ ，/ 門前/ 攤/ 了/ 麥子/ ，/ 奶奶/ 總是/ 說/ ：/ 這塊/ 醜石/ ，/ 多/ 佔/ 地面/ 呀/ ，/ 抽空/ 把/ 它/ 搬走/ 吧/ 。/ 它/ 不/ 像/ 漢/ 白玉/ 那樣/ 的/ 細膩/ ，/ 可以/ 刻字/ 雕花/ ，/ 也/ 不/ 像/ 大/ 青石/ 那樣/ 的/ 光滑/ ，/ 可以/ 供來/ 浣/ 紗捶布/ 。/ 它靜靜/ 地/ 臥/ 在/ 那/ 裏/ ，/ 院邊/ 的/ 槐/ 陰/ 沒/ 有/ 庇/ 覆/ 它/ ，/ 花兒/ 也/ 不再/ 在/ 它/ 身邊/ 生長/ 。/ 荒草/ 便/ 繁衍/ 出來/ ，/ 枝蔓/ 上下/ ，/ 慢慢/ 地/ ，/ 它/ 竟銹上/ 了/ 綠/ 苔/ 、/ 黑斑/ 。/ 我們/ 這些/ 做/ 孩子/ 的/ ，/ 也/ 討厭/ 起/ 它/ 來/ ，/ 曾合夥要/ 搬走/ 它/ ，/ 但力氣/ 又/ 不足/ ；/ 雖時/ 時咒/ 罵/ 它/ ，/ 嫌棄/ 它/ ，/ 也/ 無/ 可/ 奈何/ ，/ 只好/ 任它/ 留在/ 那/ 裏/ 了/ 。/ 終有/ 一日/ ，/ 村子/ 裏/ 來/ 了/ 一個/ 天文/ 學家/ 。/ 他/ 在/ 我家/ 門/ 前路/ 過/ ，/ 突然/ 發現/ 了/ 這塊/ 石頭/ ，/ 眼光/ 立即/ 就/ 拉直/ 了/ 。/ 他/ 再/ 沒/ 有/ 離開/ ，/ 就/ 住/ 了/ 下來/ ；/ 以/ 後/ 又/ 來/ 了/ 好些/ 人/ ，/ 都/ 說/ 這是/ 一塊/ 隕石/ ，/ 從/ 天上/ 落下/ 來/ 已/ 經有/ 二三百年/ 了/ ，/ 是/ 一件/ 了不起/ 的/ 東西/ 。/ 不久/ 便來/ 了車/ ，/ 小心翼翼/ 地將/ 它/ 運走/ 了/ 。/ 這使/ 我們/ 都/ 很/ 驚奇/ ，/ 這又怪/ 又醜/ 的/ 石頭/ ，/ 原來/ 是/ 天上/ 的/ 啊/ ！/ 它/ 補過/ 天/ ，/ 在/ 天上/ 發過熱/ 、/ 閃過/ 光/ ，/ 我們/ 的/ 先祖

### 主要詞彙

In [130]:
from itertools import groupby
from collections import Counter

counter=Counter(seg_list)
freq_lst=[list(counter.keys()),list(counter.values())]
freq_df = pd.DataFrame(np.array(freq_lst).T, columns=['Letter','wordCount'])
freq_df["wordCount"] = freq_df["wordCount"].astype(int)
sort_df = freq_df.sort_values("wordCount", ascending=False)
print(sort_df[sort_df["wordCount"] > 1])

    Letter  wordCount
7        ，         39
8        它         16
35       了         14
27       的         13
30       。         11
13       裏          7
19       也          7
11       在          6
88       、          5
96       來          5
20       不          4
17       ；          4
0        我          4
12       那          3
6       醜石          3
140     天上          3
22       是          3
99       又          3
161      ！          3
183     感到          3
90      我們          3
53      那樣          2
122     石頭          2
41      這塊          2
176      而          2
48      搬走          2
50       像          2
124     立即          2
55      可以          2
135      都          2
125      就          2
65       地          2
141     落下          2
70       沒          2
71       有          2
78      荒草          2
39       說          2
116      他          2
26       這          2
25      留在          2
3       我家          2
186     偉大          2
4       門前          2
18       誰          2


### 必讀輕聲

In [146]:
def softly_sound(passage):
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 0: 
            print(ch, py)
            
#softly_sound(passages[passage_no-1])

def highlight_softly_sound(passage):
    result = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 0: 
            result += formatmd(ch, "blue")
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_softly_sound(passages[passage_no-1])

我常常遺憾我家門前那塊醜石，它黑黝黝<span style='color:blue'>地</span>臥在那裏，牛似<span style='color:blue'>的</span>模樣；誰也不知道是什<span style='color:blue'>麼</span>時<span style='color:blue'>候</span>留在這裏<span style='color:blue'>的</span>，誰也不去理會它。只是麥收時節，門前攤<span style='color:blue'>了</span>麥<span style='color:blue'>子</span>，奶<span style='color:blue'>奶</span>總是說：這塊醜石，多佔地面<span style='color:blue'>呀</span>，抽空把它搬走<span style='color:blue'>吧</span>。它不像漢白玉那樣<span style='color:blue'>的</span>細膩，可以刻字雕花，也不像大青石那樣<span style='color:blue'>的</span>光滑，可以供來浣紗捶布。它靜靜<span style='color:blue'>地</span>臥在那裏，院邊<span style='color:blue'>的</span>槐陰沒有庇覆它，花兒也不再在它身邊生長。荒草便繁衍出來，枝蔓上下，慢慢<span style='color:blue'>地</span>，它竟銹上<span style='color:blue'>了</span>綠苔、黑斑。我<span style='color:blue'>們</span>這些做孩<span style='color:blue'>子</span><span style='color:blue'>的</span>，也討厭起它來，曾合夥要搬走它，但力<span style='color:blue'>氣</span>又不足；雖時時咒罵它，嫌棄它，也無可奈何，只好任它留在那裏<span style='color:blue'>了</span>。終有一日，村<span style='color:blue'>子</span>裏來<span style='color:blue'>了</span>一個天文學家。他在我家門前路過，突然發現<span style='color:blue'>了</span>這塊石<span style='color:blue'>頭</span>，眼光立即就拉直<span style='color:blue'>了</span>。他再沒有離開，就住<span style='color:blue'>了</span>下來；以後又來<span style='color:blue'>了</span>好些人，都說這是一塊隕石，從天上落下來已經有二三百年<span style='color:blue'>了</span>，是一件了不起<span style='color:blue'>的</span>東<span style='color:blue'>西</span>。不久便來<span style='color:blue'>了</span>車，小心翼翼<span style='color:blue'>地</span>將它運走<span style='color:blue'>了</span>。這使我<span style='color:blue'>們</span>都很驚奇，這又怪又醜<span style='color:blue'>的</span>石<span style='color:blue'>頭</span>，原來是天上<span style='color:blue'>的</span><span style='color:blue'>啊</span>！它補過天，在天上發過熱、閃過光，我<span style='color:blue'>們</span><span style='color:blue'>的</span>先祖或許仰望過它，它給<span style='color:blue'>了</span>他<span style='color:blue'>們</span>光明、向往、憧憬；而它落下來<span style='color:blue'>了</span>，在污土裏，荒草裏，一躺就//是幾百年<span style='color:blue'>了</span>！我感到自己<span style='color:blue'>的</span>無知，也感到<span style='color:blue'>了</span>醜石<span style='color:blue'>的</span>偉大，我甚至怨恨它這<span style='color:blue'>麼</span>多年竟會默默<span style='color:blue'>地</span>忍受<span style='color:blue'>着</span>這一切！而我又立即深深<span style='color:blue'>地</span>感到它那種不屈於誤解、寂寞<span style='color:blue'>的</span>生存<span style='color:blue'>的</span>偉大。

### 去聲變調

In [145]:
def third_sound_steak(passage):
    cur = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 3: 
            cur += ch
        else:
            if len(cur) > 1:
                print(cur)
            cur = ""
            
#third_sound_steak(passages[passage_no-1])

def highlight_third_sound_steak(passage):
    result = ""
    cur = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 3: 
            cur += ch
        else:
            if len(cur) > 1:
                result += formatmd(cur, 'blue')
            cur = ""
            result += formatmd(ch)
            
    display(Markdown(result)) 
            
highlight_third_sound_steak(passages[passage_no-1])

<span style='color:None'>常</span><span style='color:None'>常</span><span style='color:None'>遺</span><span style='color:None'>憾</span><span style='color:None'>家</span><span style='color:None'>門</span><span style='color:None'>前</span><span style='color:None'>那</span><span style='color:None'>塊</span><span style='color:None'>石</span><span style='color:None'>，</span><span style='color:None'>它</span><span style='color:None'>黑</span><span style='color:blue'>黝黝</span><span style='color:None'>地</span><span style='color:None'>臥</span><span style='color:None'>在</span><span style='color:None'>那</span><span style='color:None'>，</span><span style='color:None'>牛</span><span style='color:None'>似</span><span style='color:None'>的</span><span style='color:None'>模</span><span style='color:None'>樣</span><span style='color:None'>；</span><span style='color:None'>誰</span><span style='color:None'>不</span><span style='color:None'>知</span><span style='color:None'>道</span><span style='color:None'>是</span><span style='color:None'>什</span><span style='color:None'>麼</span><span style='color:None'>時</span><span style='color:None'>候</span><span style='color:None'>留</span><span style='color:None'>在</span><span style='color:None'>這</span><span style='color:None'>的</span><span style='color:None'>，</span><span style='color:None'>誰</span><span style='color:None'>不</span><span style='color:None'>去</span><span style='color:None'>會</span><span style='color:None'>它</span><span style='color:None'>。</span><span style='color:None'>是</span><span style='color:None'>麥</span><span style='color:None'>收</span><span style='color:None'>時</span><span style='color:None'>節</span><span style='color:None'>，</span><span style='color:None'>門</span><span style='color:None'>前</span><span style='color:None'>攤</span><span style='color:None'>了</span><span style='color:None'>麥</span><span style='color:None'>子</span><span style='color:None'>，</span><span style='color:None'>奶</span><span style='color:None'>是</span><span style='color:None'>說</span><span style='color:None'>：</span><span style='color:None'>這</span><span style='color:None'>塊</span><span style='color:None'>石</span><span style='color:None'>，</span><span style='color:None'>多</span><span style='color:None'>佔</span><span style='color:None'>地</span><span style='color:None'>面</span><span style='color:None'>呀</span><span style='color:None'>，</span><span style='color:None'>抽</span><span style='color:None'>空</span><span style='color:None'>它</span><span style='color:None'>搬</span><span style='color:None'>吧</span><span style='color:None'>。</span><span style='color:None'>它</span><span style='color:None'>不</span><span style='color:None'>像</span><span style='color:None'>漢</span><span style='color:None'>白</span><span style='color:None'>玉</span><span style='color:None'>那</span><span style='color:None'>樣</span><span style='color:None'>的</span><span style='color:None'>細</span><span style='color:None'>膩</span><span style='color:None'>，</span><span style='color:blue'>可以</span><span style='color:None'>刻</span><span style='color:None'>字</span><span style='color:None'>雕</span><span style='color:None'>花</span><span style='color:None'>，</span><span style='color:None'>不</span><span style='color:None'>像</span><span style='color:None'>大</span><span style='color:None'>青</span><span style='color:None'>石</span><span style='color:None'>那</span><span style='color:None'>樣</span><span style='color:None'>的</span><span style='color:None'>光</span><span style='color:None'>滑</span><span style='color:None'>，</span><span style='color:blue'>可以</span><span style='color:None'>供</span><span style='color:None'>來</span><span style='color:None'>浣</span><span style='color:None'>紗</span><span style='color:None'>捶</span><span style='color:None'>布</span><span style='color:None'>。</span><span style='color:None'>它</span><span style='color:None'>靜</span><span style='color:None'>靜</span><span style='color:None'>地</span><span style='color:None'>臥</span><span style='color:None'>在</span><span style='color:None'>那</span><span style='color:None'>，</span><span style='color:None'>院</span><span style='color:None'>邊</span><span style='color:None'>的</span><span style='color:None'>槐</span><span style='color:None'>陰</span><span style='color:None'>沒</span><span style='color:None'>庇</span><span style='color:None'>覆</span><span style='color:None'>它</span><span style='color:None'>，</span><span style='color:None'>花</span><span style='color:None'>兒</span><span style='color:None'>不</span><span style='color:None'>再</span><span style='color:None'>在</span><span style='color:None'>它</span><span style='color:None'>身</span><span style='color:None'>邊</span><span style='color:None'>生</span><span style='color:None'>。</span><span style='color:None'>荒</span><span style='color:None'>便</span><span style='color:None'>繁</span><span style='color:None'>出</span><span style='color:None'>來</span><span style='color:None'>，</span><span style='color:None'>枝</span><span style='color:None'>蔓</span><span style='color:None'>上</span><span style='color:None'>下</span><span style='color:None'>，</span><span style='color:None'>慢</span><span style='color:None'>慢</span><span style='color:None'>地</span><span style='color:None'>，</span><span style='color:None'>它</span><span style='color:None'>竟</span><span style='color:None'>銹</span><span style='color:None'>上</span><span style='color:None'>了</span><span style='color:None'>綠</span><span style='color:None'>苔</span><span style='color:None'>、</span><span style='color:None'>黑</span><span style='color:None'>斑</span><span style='color:None'>。</span><span style='color:None'>們</span><span style='color:None'>這</span><span style='color:None'>些</span><span style='color:None'>做</span><span style='color:None'>孩</span><span style='color:None'>子</span><span style='color:None'>的</span><span style='color:None'>，</span><span style='color:blue'>也討</span><span style='color:None'>厭</span><span style='color:None'>它</span><span style='color:None'>來</span><span style='color:None'>，</span><span style='color:None'>曾</span><span style='color:None'>合</span><span style='color:None'>要</span><span style='color:None'>搬</span><span style='color:None'>它</span><span style='color:None'>，</span><span style='color:None'>但</span><span style='color:None'>力</span><span style='color:None'>氣</span><span style='color:None'>又</span><span style='color:None'>不</span><span style='color:None'>足</span><span style='color:None'>；</span><span style='color:None'>雖</span><span style='color:None'>時</span><span style='color:None'>時</span><span style='color:None'>咒</span><span style='color:None'>罵</span><span style='color:None'>它</span><span style='color:None'>，</span><span style='color:None'>嫌</span><span style='color:None'>棄</span><span style='color:None'>它</span><span style='color:None'>，</span><span style='color:None'>無</span><span style='color:None'>奈</span><span style='color:None'>何</span><span style='color:None'>，</span><span style='color:blue'>只好</span><span style='color:None'>任</span><span style='color:None'>它</span><span style='color:None'>留</span><span style='color:None'>在</span><span style='color:None'>那</span><span style='color:None'>了</span><span style='color:None'>。</span><span style='color:None'>終</span><span style='color:None'>一</span><span style='color:None'>日</span><span style='color:None'>，</span><span style='color:None'>村</span><span style='color:None'>子</span><span style='color:None'>來</span><span style='color:None'>了</span><span style='color:None'>一</span><span style='color:None'>個</span><span style='color:None'>天</span><span style='color:None'>文</span><span style='color:None'>學</span><span style='color:None'>家</span><span style='color:None'>。</span><span style='color:None'>他</span><span style='color:None'>在</span><span style='color:None'>家</span><span style='color:None'>門</span><span style='color:None'>前</span><span style='color:None'>路</span><span style='color:None'>過</span><span style='color:None'>，</span><span style='color:None'>突</span><span style='color:None'>然</span><span style='color:None'>發</span><span style='color:None'>現</span><span style='color:None'>了</span><span style='color:None'>這</span><span style='color:None'>塊</span><span style='color:None'>石</span><span style='color:None'>頭</span><span style='color:None'>，</span><span style='color:None'>光</span><span style='color:None'>立</span><span style='color:None'>即</span><span style='color:None'>就</span><span style='color:None'>拉</span><span style='color:None'>直</span><span style='color:None'>了</span><span style='color:None'>。</span><span style='color:None'>他</span><span style='color:None'>再</span><span style='color:None'>沒</span><span style='color:None'>離</span><span style='color:None'>開</span><span style='color:None'>，</span><span style='color:None'>就</span><span style='color:None'>住</span><span style='color:None'>了</span><span style='color:None'>下</span><span style='color:None'>來</span><span style='color:None'>；</span><span style='color:None'>後</span><span style='color:None'>又</span><span style='color:None'>來</span><span style='color:None'>了</span><span style='color:None'>些</span><span style='color:None'>人</span><span style='color:None'>，</span><span style='color:None'>都</span><span style='color:None'>說</span><span style='color:None'>這</span><span style='color:None'>是</span><span style='color:None'>一</span><span style='color:None'>塊</span><span style='color:None'>石</span><span style='color:None'>，</span><span style='color:None'>從</span><span style='color:None'>天</span><span style='color:None'>上</span><span style='color:None'>落</span><span style='color:None'>下</span><span style='color:None'>來</span><span style='color:None'>經</span><span style='color:None'>二</span><span style='color:None'>三</span><span style='color:None'>年</span><span style='color:None'>了</span><span style='color:None'>，</span><span style='color:None'>是</span><span style='color:None'>一</span><span style='color:None'>件</span><span style='color:None'>不</span><span style='color:None'>的</span><span style='color:None'>東</span><span style='color:None'>西</span><span style='color:None'>。</span><span style='color:None'>不</span><span style='color:None'>便</span><span style='color:None'>來</span><span style='color:None'>了</span><span style='color:None'>車</span><span style='color:None'>，</span><span style='color:None'>心</span><span style='color:None'>翼</span><span style='color:None'>翼</span><span style='color:None'>地</span><span style='color:None'>將</span><span style='color:None'>它</span><span style='color:None'>運</span><span style='color:None'>了</span><span style='color:None'>。</span><span style='color:None'>這</span><span style='color:blue'>使我</span><span style='color:None'>們</span><span style='color:None'>都</span><span style='color:None'>驚</span><span style='color:None'>奇</span><span style='color:None'>，</span><span style='color:None'>這</span><span style='color:None'>又</span><span style='color:None'>怪</span><span style='color:None'>又</span><span style='color:None'>的</span><span style='color:None'>石</span><span style='color:None'>頭</span><span style='color:None'>，</span><span style='color:None'>原</span><span style='color:None'>來</span><span style='color:None'>是</span><span style='color:None'>天</span><span style='color:None'>上</span><span style='color:None'>的</span><span style='color:None'>啊</span><span style='color:None'>！</span><span style='color:None'>它</span><span style='color:None'>過</span><span style='color:None'>天</span><span style='color:None'>，</span><span style='color:None'>在</span><span style='color:None'>天</span><span style='color:None'>上</span><span style='color:None'>發</span><span style='color:None'>過</span><span style='color:None'>熱</span><span style='color:None'>、</span><span style='color:None'>過</span><span style='color:None'>光</span><span style='color:None'>，</span><span style='color:None'>們</span><span style='color:None'>的</span><span style='color:None'>先</span><span style='color:None'>或</span><span style='color:blue'>許仰</span><span style='color:None'>望</span><span style='color:None'>過</span><span style='color:None'>它</span><span style='color:None'>，</span><span style='color:None'>它</span><span style='color:None'>了</span><span style='color:None'>他</span><span style='color:None'>們</span><span style='color:None'>光</span><span style='color:None'>明</span><span style='color:None'>、</span><span style='color:None'>向</span><span style='color:None'>、</span><span style='color:None'>憧</span><span style='color:None'>；</span><span style='color:None'>而</span><span style='color:None'>它</span><span style='color:None'>落</span><span style='color:None'>下</span><span style='color:None'>來</span><span style='color:None'>了</span><span style='color:None'>，</span><span style='color:None'>在</span><span style='color:None'>污</span><span style='color:blue'>土裏</span><span style='color:None'>，</span><span style='color:None'>荒</span><span style='color:blue'>草裏</span><span style='color:None'>，</span><span style='color:None'>一</span><span style='color:None'>就</span><span style='color:None'>//</span><span style='color:None'>是</span><span style='color:blue'>幾百</span><span style='color:None'>年</span><span style='color:None'>了</span><span style='color:None'>！</span><span style='color:blue'>我感</span><span style='color:None'>到</span><span style='color:None'>自</span><span style='color:None'>的</span><span style='color:None'>無</span><span style='color:None'>知</span><span style='color:None'>，</span><span style='color:blue'>也感</span><span style='color:None'>到</span><span style='color:None'>了</span><span style='color:None'>石</span><span style='color:None'>的</span><span style='color:None'>大</span><span style='color:None'>，</span><span style='color:None'>甚</span><span style='color:None'>至</span><span style='color:None'>怨</span><span style='color:None'>恨</span><span style='color:None'>它</span><span style='color:None'>這</span><span style='color:None'>麼</span><span style='color:None'>多</span><span style='color:None'>年</span><span style='color:None'>竟</span><span style='color:None'>會</span><span style='color:None'>默</span><span style='color:None'>默</span><span style='color:None'>地</span><span style='color:None'>受</span><span style='color:None'>着</span><span style='color:None'>這</span><span style='color:None'>一</span><span style='color:None'>切</span><span style='color:None'>！</span><span style='color:None'>而</span><span style='color:None'>又</span><span style='color:None'>立</span><span style='color:None'>即</span><span style='color:None'>深</span><span style='color:None'>深</span><span style='color:None'>地</span><span style='color:None'>到</span><span style='color:None'>它</span><span style='color:None'>那</span><span style='color:None'>不</span><span style='color:None'>屈</span><span style='color:None'>於</span><span style='color:None'>誤</span><span style='color:None'>、</span><span style='color:None'>寂</span><span style='color:None'>寞</span><span style='color:None'>的</span><span style='color:None'>生</span><span style='color:None'>存</span><span style='color:None'>的</span><span style='color:None'>大</span><span style='color:None'>。</span>

### 一不變調

In [184]:
def highlight_yi_bu(passage):
    result = ""
    cur = ""
    for py, ch in passage:
        if (ch == '一' and py != 'yī') or (ch == '不' and py != 'bù'):
            result += formatmd(ch, 'blue')
        elif (ch == '一' and py == 'yī') or (ch == '不' and py == 'bù'):
            result += formatmd(ch, 'green')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_yi_bu(passages[passage_no-1])

我常常遺憾我家門前那塊醜石，它黑黝黝地臥在那裏，牛似的模樣；誰也<span style='color:green'>不</span>知道是什麼時候留在這裏的，誰也<span style='color:blue'>不</span>去理會它。只是麥收時節，門前攤了麥子，奶奶總是說：這塊醜石，多佔地面呀，抽空把它搬走吧。它<span style='color:blue'>不</span>像漢白玉那樣的細膩，可以刻字雕花，也<span style='color:blue'>不</span>像大青石那樣的光滑，可以供來浣紗捶布。它靜靜地臥在那裏，院邊的槐陰沒有庇覆它，花兒也<span style='color:blue'>不</span>再在它身邊生長。荒草便繁衍出來，枝蔓上下，慢慢地，它竟銹上了綠苔、黑斑。我們這些做孩子的，也討厭起它來，曾合夥要搬走它，但力氣又<span style='color:green'>不</span>足；雖時時咒罵它，嫌棄它，也無可奈何，只好任它留在那裏了。終有<span style='color:blue'>一</span>日，村子裏來了<span style='color:blue'>一</span>個天文學家。他在我家門前路過，突然發現了這塊石頭，眼光立即就拉直了。他再沒有離開，就住了下來；以後又來了好些人，都說這是<span style='color:blue'>一</span>塊隕石，從天上落下來已經有二三百年了，是<span style='color:blue'>一</span>件了<span style='color:green'>不</span>起的東西。<span style='color:green'>不</span>久便來了車，小心翼翼地將它運走了。這使我們都很驚奇，這又怪又醜的石頭，原來是天上的啊！它補過天，在天上發過熱、閃過光，我們的先祖或許仰望過它，它給了他們光明、向往、憧憬；而它落下來了，在污土裏，荒草裏，<span style='color:blue'>一</span>躺就//是幾百年了！我感到自己的無知，也感到了醜石的偉大，我甚至怨恨它這麼多年竟會默默地忍受着這<span style='color:blue'>一</span>切！而我又立即深深地感到它那種<span style='color:green'>不</span>屈於誤解、寂寞的生存的偉大。

### 平舌音

In [149]:
def zcs_in_passage(passage): 
    for py, ch in passage:
        if((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            print(ch, py)
            
#zcs_in_passage(passages[passage_no-1])

def highlight_zcs_in_passage(passage):
    result = ""
    for py, ch in passage:
        if((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zcs_in_passage(passages[passage_no-1])


我常常遺憾我家門前那塊醜石，它黑黝黝地臥<span style='color:blue'>在</span>那裏，牛似的模樣；誰也不知道是什麼時候留<span style='color:blue'>在</span>這裏的，誰也不去理會它。只是麥收時節，門前攤了麥<span style='color:blue'>子</span>，奶奶<span style='color:blue'>總</span>是說：這塊醜石，多佔地面呀，抽空把它搬<span style='color:blue'>走</span>吧。它不像漢白玉那樣的細膩，可以刻<span style='color:blue'>字</span>雕花，也不像大青石那樣的光滑，可以供來浣紗捶布。它靜靜地臥<span style='color:blue'>在</span>那裏，院邊的槐陰沒有庇覆它，花兒也不<span style='color:blue'>再</span><span style='color:blue'>在</span>它身邊生長。荒<span style='color:blue'>草</span>便繁衍出來，枝蔓上下，慢慢地，它竟銹上了綠苔、黑斑。我們這些<span style='color:blue'>做</span>孩<span style='color:blue'>子</span>的，也討厭起它來，<span style='color:blue'>曾</span>合夥要搬<span style='color:blue'>走</span>它，但力氣又不<span style='color:blue'>足</span>；<span style='color:blue'>雖</span>時時咒罵它，嫌棄它，也無可奈何，只好任它留<span style='color:blue'>在</span>那裏了。終有一日，<span style='color:blue'>村</span><span style='color:blue'>子</span>裏來了一個天文學家。他<span style='color:blue'>在</span>我家門前路過，突然發現了這塊石頭，眼光立即就拉直了。他<span style='color:blue'>再</span>沒有離開，就住了下來；以後又來了好些人，都說這是一塊隕石，<span style='color:blue'>從</span>天上落下來已經有二<span style='color:blue'>三</span>百年了，是一件了不起的東西。不久便來了車，小心翼翼地將它運<span style='color:blue'>走</span>了。這使我們都很驚奇，這又怪又醜的石頭，原來是天上的啊！它補過天，<span style='color:blue'>在</span>天上發過熱、閃過光，我們的先<span style='color:blue'>祖</span>或許仰望過它，它給了他們光明、向往、憧憬；而它落下來了，<span style='color:blue'>在</span>污土裏，荒<span style='color:blue'>草</span>裏，一躺就//是幾百年了！我感到<span style='color:blue'>自</span>己的無知，也感到了醜石的偉大，我甚至怨恨它這麼多年竟會默默地忍受着這一切！而我又立即深深地感到它那種不屈於誤解、寂寞的生<span style='color:blue'>存</span>的偉大。

### 翹舌音

In [150]:
def zhchsh_in_passage(passage): 
    for py, ch in passage:
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            print(ch, py)
            
#zhchsh_in_passage(passages[passage_no-1])

def highlight_zhchsh_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zhchsh_in_passage(passages[passage_no-1])

我<span style='color:blue'>常</span><span style='color:blue'>常</span>遺憾我家門前那塊<span style='color:blue'>醜</span><span style='color:blue'>石</span>，它黑黝黝地臥在那裏，牛<span style='color:blue'>似</span>的模樣；<span style='color:blue'>誰</span>也不<span style='color:blue'>知</span>道<span style='color:blue'>是</span><span style='color:blue'>什</span>麼<span style='color:blue'>時</span>候留在<span style='color:blue'>這</span>裏的，<span style='color:blue'>誰</span>也不去理會它。<span style='color:blue'>只</span><span style='color:blue'>是</span>麥<span style='color:blue'>收</span><span style='color:blue'>時</span>節，門前攤了麥子，奶奶總<span style='color:blue'>是</span><span style='color:blue'>說</span>：<span style='color:blue'>這</span>塊<span style='color:blue'>醜</span><span style='color:blue'>石</span>，多<span style='color:blue'>佔</span>地面呀，<span style='color:blue'>抽</span>空把它搬走吧。它不像漢白玉那樣的細膩，可以刻字雕花，也不像大青<span style='color:blue'>石</span>那樣的光滑，可以供來浣<span style='color:blue'>紗</span><span style='color:blue'>捶</span>布。它靜靜地臥在那裏，院邊的槐陰沒有庇覆它，花兒也不再在它<span style='color:blue'>身</span>邊<span style='color:blue'>生</span><span style='color:blue'>長</span>。荒草便繁衍<span style='color:blue'>出</span>來，<span style='color:blue'>枝</span>蔓<span style='color:blue'>上</span>下，慢慢地，它竟銹<span style='color:blue'>上</span>了綠苔、黑斑。我們<span style='color:blue'>這</span>些做孩子的，也討厭起它來，曾合夥要搬走它，但力氣又不足；雖<span style='color:blue'>時</span><span style='color:blue'>時</span><span style='color:blue'>咒</span>罵它，嫌棄它，也無可奈何，<span style='color:blue'>只</span>好任它留在那裏了。<span style='color:blue'>終</span>有一日，村子裏來了一個天文學家。他在我家門前路過，突然發現了<span style='color:blue'>這</span>塊<span style='color:blue'>石</span>頭，眼光立即就拉<span style='color:blue'>直</span>了。他再沒有離開，就<span style='color:blue'>住</span>了下來；以後又來了好些人，都<span style='color:blue'>說</span><span style='color:blue'>這</span><span style='color:blue'>是</span>一塊隕<span style='color:blue'>石</span>，從天<span style='color:blue'>上</span>落下來已經有二三百年了，<span style='color:blue'>是</span>一件了不起的東西。不久便來了<span style='color:blue'>車</span>，小心翼翼地將它運走了。<span style='color:blue'>這</span><span style='color:blue'>使</span>我們都很驚奇，<span style='color:blue'>這</span>又怪又<span style='color:blue'>醜</span>的<span style='color:blue'>石</span>頭，原來<span style='color:blue'>是</span>天<span style='color:blue'>上</span>的啊！它補過天，在天<span style='color:blue'>上</span>發過熱、<span style='color:blue'>閃</span>過光，我們的先祖或許仰望過它，它給了他們光明、向往、<span style='color:blue'>憧</span>憬；而它落下來了，在污土裏，荒草裏，一躺就//<span style='color:blue'>是</span>幾百年了！我感到自己的無<span style='color:blue'>知</span>，也感到了<span style='color:blue'>醜</span><span style='color:blue'>石</span>的偉大，我<span style='color:blue'>甚</span><span style='color:blue'>至</span>怨恨它<span style='color:blue'>這</span>麼多年竟會默默地忍<span style='color:blue'>受</span><span style='color:blue'>着</span><span style='color:blue'>這</span>一切！而我又立即<span style='color:blue'>深</span><span style='color:blue'>深</span>地感到它那<span style='color:blue'>種</span>不屈於誤解、寂寞的<span style='color:blue'>生</span>存的偉大。

In [174]:
def highlight_zcs_zhchsh_in_passage(passage):
    result = ""
    for py, ch in passage:
        if ((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            result += formatmd(ch, 'blue')
        elif py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            result += formatmd(ch, 'green')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zcs_zhchsh_in_passage(passages[passage_no-1])

我<span style='color:green'>常</span><span style='color:green'>常</span>遺憾我家門前那塊<span style='color:green'>醜</span><span style='color:green'>石</span>，它黑黝黝地臥<span style='color:blue'>在</span>那裏，牛<span style='color:green'>似</span>的模樣；<span style='color:green'>誰</span>也不<span style='color:green'>知</span>道<span style='color:green'>是</span><span style='color:green'>什</span>麼<span style='color:green'>時</span>候留<span style='color:blue'>在</span><span style='color:green'>這</span>裏的，<span style='color:green'>誰</span>也不去理會它。<span style='color:green'>只</span><span style='color:green'>是</span>麥<span style='color:green'>收</span><span style='color:green'>時</span>節，門前攤了麥<span style='color:blue'>子</span>，奶奶<span style='color:blue'>總</span><span style='color:green'>是</span><span style='color:green'>說</span>：<span style='color:green'>這</span>塊<span style='color:green'>醜</span><span style='color:green'>石</span>，多<span style='color:green'>佔</span>地面呀，<span style='color:green'>抽</span>空把它搬<span style='color:blue'>走</span>吧。它不像漢白玉那樣的細膩，可以刻<span style='color:blue'>字</span>雕花，也不像大青<span style='color:green'>石</span>那樣的光滑，可以供來浣<span style='color:green'>紗</span><span style='color:green'>捶</span>布。它靜靜地臥<span style='color:blue'>在</span>那裏，院邊的槐陰沒有庇覆它，花兒也不<span style='color:blue'>再</span><span style='color:blue'>在</span>它<span style='color:green'>身</span>邊<span style='color:green'>生</span><span style='color:green'>長</span>。荒<span style='color:blue'>草</span>便繁衍<span style='color:green'>出</span>來，<span style='color:green'>枝</span>蔓<span style='color:green'>上</span>下，慢慢地，它竟銹<span style='color:green'>上</span>了綠苔、黑斑。我們<span style='color:green'>這</span>些<span style='color:blue'>做</span>孩<span style='color:blue'>子</span>的，也討厭起它來，<span style='color:blue'>曾</span>合夥要搬<span style='color:blue'>走</span>它，但力氣又不<span style='color:blue'>足</span>；<span style='color:blue'>雖</span><span style='color:green'>時</span><span style='color:green'>時</span><span style='color:green'>咒</span>罵它，嫌棄它，也無可奈何，<span style='color:green'>只</span>好任它留<span style='color:blue'>在</span>那裏了。<span style='color:green'>終</span>有一日，<span style='color:blue'>村</span><span style='color:blue'>子</span>裏來了一個天文學家。他<span style='color:blue'>在</span>我家門前路過，突然發現了<span style='color:green'>這</span>塊<span style='color:green'>石</span>頭，眼光立即就拉<span style='color:green'>直</span>了。他<span style='color:blue'>再</span>沒有離開，就<span style='color:green'>住</span>了下來；以後又來了好些人，都<span style='color:green'>說</span><span style='color:green'>這</span><span style='color:green'>是</span>一塊隕<span style='color:green'>石</span>，<span style='color:blue'>從</span>天<span style='color:green'>上</span>落下來已經有二<span style='color:blue'>三</span>百年了，<span style='color:green'>是</span>一件了不起的東西。不久便來了<span style='color:green'>車</span>，小心翼翼地將它運<span style='color:blue'>走</span>了。<span style='color:green'>這</span><span style='color:green'>使</span>我們都很驚奇，<span style='color:green'>這</span>又怪又<span style='color:green'>醜</span>的<span style='color:green'>石</span>頭，原來<span style='color:green'>是</span>天<span style='color:green'>上</span>的啊！它補過天，<span style='color:blue'>在</span>天<span style='color:green'>上</span>發過熱、<span style='color:green'>閃</span>過光，我們的先<span style='color:blue'>祖</span>或許仰望過它，它給了他們光明、向往、<span style='color:green'>憧</span>憬；而它落下來了，<span style='color:blue'>在</span>污土裏，荒<span style='color:blue'>草</span>裏，一躺就//<span style='color:green'>是</span>幾百年了！我感到<span style='color:blue'>自</span>己的無<span style='color:green'>知</span>，也感到了<span style='color:green'>醜</span><span style='color:green'>石</span>的偉大，我<span style='color:green'>甚</span><span style='color:green'>至</span>怨恨它<span style='color:green'>這</span>麼多年竟會默默地忍<span style='color:green'>受</span><span style='color:green'>着</span><span style='color:green'>這</span>一切！而我又立即<span style='color:green'>深</span><span style='color:green'>深</span>地感到它那<span style='color:green'>種</span>不屈於誤解、寂寞的<span style='color:green'>生</span><span style='color:blue'>存</span>的偉大。

### 多音字

In [153]:
def multiple_sound_in_passage(passage):
    for py, ch in passage:
        if ch in char_dict:
            p_dict = char_dict[ch]
            if len(p_dict.keys()) > 1:
                print(ch, py, list(p_dict.keys()))

# multiple_sound_in_passage(passages[passage_no-1])

def highlight_multiple_sound_in_passage(passage):
    result = ""
    for py, ch in passage:
        if ch in char_dict:
            p_dict = char_dict[ch]
            valid_cnt = 0
            for key in p_dict.keys():
                if character_tone(key) >0:
                    valid_cnt = valid_cnt+1
            
            if valid_cnt > 1:
                result += formatmd(ch, 'red')
            else:
                result += ch
            
    display(Markdown(result)) 
            
highlight_multiple_sound_in_passage(passages[passage_no-1])

我常常遺憾我家門前那塊醜<span style='color:red'>石</span>它黑黝黝地在那裏牛<span style='color:red'>似</span><span style='color:red'>的</span><span style='color:red'>模</span>樣誰也不知道是<span style='color:red'>什</span>麼時候留在這裏<span style='color:red'>的</span>誰也不去理<span style='color:red'>會</span>它<span style='color:red'>只</span>是麥收時節門前攤了麥子奶奶總是說這塊醜<span style='color:red'>石</span>多地面抽<span style='color:red'>空</span><span style='color:red'>把</span>它搬走吧它不像漢白玉那樣<span style='color:red'>的</span>細膩可以刻字雕花也不像<span style='color:red'>大</span>青<span style='color:red'>石</span>那樣<span style='color:red'>的</span>光滑可以<span style='color:red'>供</span>來紗捶布它靜靜地在那裏院邊<span style='color:red'>的</span>槐陰<span style='color:red'>沒</span>有庇覆它花兒也不再在它身邊生<span style='color:red'>長</span>荒草<span style='color:red'>便</span>繁衍出來枝<span style='color:red'>蔓</span><span style='color:red'>上</span>下慢慢地它竟銹<span style='color:red'>上</span>了<span style='color:red'>綠</span><span style='color:red'>苔</span>黑斑我們這些做孩子<span style='color:red'>的</span>也討厭起它來<span style='color:red'>曾</span>合夥<span style='color:red'>要</span>搬走它但力氣又不足雖時時咒罵它嫌棄它也無可奈何<span style='color:red'>只</span><span style='color:red'>好</span><span style='color:red'>任</span>它留在那裏了終有一日村子裏來了一個天文學家他在我家門前路過突然<span style='color:red'>發</span>現了這塊<span style='color:red'>石</span>頭眼光立即就<span style='color:red'>拉</span>直了他再<span style='color:red'>沒</span>有離開就住了下來以後又來了<span style='color:red'>好</span>些人<span style='color:red'>都</span>說這是一塊隕<span style='color:red'>石</span>從天<span style='color:red'>上</span><span style='color:red'>落</span>下來已經有二三百年了是一件了不起<span style='color:red'>的</span>東西不久<span style='color:red'>便</span>來了<span style='color:red'>車</span>小心翼翼地<span style='color:red'>將</span>它運走了這使我們<span style='color:red'>都</span>很驚奇這又怪又醜<span style='color:red'>的</span><span style='color:red'>石</span>頭原來是天<span style='color:red'>上</span><span style='color:red'>的</span>它補過天在天<span style='color:red'>上</span><span style='color:red'>發</span>過熱閃過光我們<span style='color:red'>的</span>先祖或許仰望過它它<span style='color:red'>給</span>了他們光明向往憧憬而它<span style='color:red'>落</span>下來了在污土裏荒草裏一躺就是<span style='color:red'>幾</span>百年了我感到自己<span style='color:red'>的</span>無知也感到了醜<span style='color:red'>石</span><span style='color:red'>的</span>偉<span style='color:red'>大</span>我甚至怨恨它這麼多年竟<span style='color:red'>會</span>默默地忍受這一<span style='color:red'>切</span>而我又立即深深地感到它那<span style='color:red'>種</span>不屈於誤<span style='color:red'>解</span>寂寞<span style='color:red'>的</span>生存<span style='color:red'>的</span>偉<span style='color:red'>大</span>

In [158]:
char_dict['切']

{'qiè': [[('密', 'mì'), ('切', 'qiè')],
  [('迫', 'pò'), ('切', 'qiè')],
  [('切', 'qiè'), ('實', 'shí')],
  [('親', 'qīn'), ('切', 'qiè')],
  [('確', 'què'), ('切', 'qiè')],
  [('一', 'yī'), ('切', 'qiè')],
  [('懇', 'kěn'), ('切', 'qiè')],
  [('切', 'qiè'), ('合', 'hé')],
  [('切', 'qiè'), ('忌', 'jì')],
  [('切', 'qiè'), ('身', 'shēn')],
  [('熱', 'rè'), ('切', 'qiè')],
  [('深', 'shēn'), ('切', 'qiè')],
  [('殷', 'yīn'), ('切', 'qiè')],
  [('真', 'zhēn'), ('切', 'qiè')]],
 'qiē': [[('切', 'qiē')],
  [('切', 'qiē')],
  [('關', 'guān'), ('切', 'qiē')],
  [('急', 'jí'), ('切', 'qiē')],
  [('切', 'qiē'), ('除', 'chú')],
  [('切', 'qiē'), ('磋', 'cuō')],
  [('切', 'qiē'), ('點', 'diǎn')],
  [('切', 'qiē'), ('割', 'gē')],
  [('切', 'qiē'), ('口', 'kǒu')],
  [('切', 'qiē'), ('麵', 'miàn')],
  [('切', 'qiē'), ('片', 'piàn')],
  [('切', 'qiē'), ('綫', 'xiàn')],
  [('貼', 'tiē'), ('切', 'qiē')],
  [('心', 'xīn'), ('切', 'qiē')]]}

### 兒化音

In [161]:
def r_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("r"):
            print(ch, py)
            
#r_in_passage(passages[passage_no-1])

def highlight_r_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("r"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_r_in_passage(passages[passage_no-1])

我常常遺憾我家門前那塊醜石，它黑黝黝地臥在那裏，牛似的模樣；誰也不知道是什麼時候留在這裏的，誰也不去理會它。只是麥收時節，門前攤了麥子，奶奶總是說：這塊醜石，多佔地面呀，抽空把它搬走吧。它不像漢白玉那樣的細膩，可以刻字雕花，也不像大青石那樣的光滑，可以供來浣紗捶布。它靜靜地臥在那裏，院邊的槐陰沒有庇覆它，花<span style='color:red'>兒</span>也不再在它身邊生長。荒草便繁衍出來，枝蔓上下，慢慢地，它竟銹上了綠苔、黑斑。我們這些做孩子的，也討厭起它來，曾合夥要搬走它，但力氣又不足；雖時時咒罵它，嫌棄它，也無可奈何，只好任它留在那裏了。終有一日，村子裏來了一個天文學家。他在我家門前路過，突然發現了這塊石頭，眼光立即就拉直了。他再沒有離開，就住了下來；以後又來了好些人，都說這是一塊隕石，從天上落下來已經有<span style='color:red'>二</span>三百年了，是一件了不起的東西。不久便來了車，小心翼翼地將它運走了。這使我們都很驚奇，這又怪又醜的石頭，原來是天上的啊！它補過天，在天上發過熱、閃過光，我們的先祖或許仰望過它，它給了他們光明、向往、憧憬；<span style='color:red'>而</span>它落下來了，在污土裏，荒草裏，一躺就//是幾百年了！我感到自己的無知，也感到了醜石的偉大，我甚至怨恨它這麼多年竟會默默地忍受着這一切！<span style='color:red'>而</span>我又立即深深地感到它那種不屈於誤解、寂寞的生存的偉大。

### 前鼻音

In [166]:
def n_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("n"):
            print(ch, py)
            
#n_in_passage(passages[passage_no-1])

def highlight_n_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("n"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_n_in_passage(passages[passage_no-1])

我常常遺<span style='color:red'>憾</span>我家<span style='color:red'>門</span><span style='color:red'>前</span>那塊醜石，它黑黝黝地臥在那裏，牛似的模樣；誰也不知道是<span style='color:red'>什</span>麼時候留在這裏的，誰也不去理會它。只是麥收時節，<span style='color:red'>門</span><span style='color:red'>前</span><span style='color:red'>攤</span>了麥子，奶奶總是說：這塊醜石，多<span style='color:red'>佔</span>地<span style='color:red'>面</span>呀，抽空把它<span style='color:red'>搬</span>走吧。它不像<span style='color:red'>漢</span>白玉那樣的細膩，可以刻字雕花，也不像大青石那樣的光滑，可以供來<span style='color:red'>浣</span>紗捶布。它靜靜地臥在那裏，<span style='color:red'>院</span><span style='color:red'>邊</span>的槐<span style='color:red'>陰</span>沒有庇覆它，花兒也不再在它<span style='color:red'>身</span><span style='color:red'>邊</span>生長。荒草<span style='color:red'>便</span><span style='color:red'>繁</span><span style='color:red'>衍</span>出來，枝<span style='color:red'>蔓</span>上下，<span style='color:red'>慢</span><span style='color:red'>慢</span>地，它竟銹上了綠苔、黑<span style='color:red'>斑</span>。我<span style='color:red'>們</span>這些做孩子的，也討<span style='color:red'>厭</span>起它來，曾合夥要<span style='color:red'>搬</span>走它，<span style='color:red'>但</span>力氣又不足；雖時時咒罵它，<span style='color:red'>嫌</span>棄它，也無可奈何，只好<span style='color:red'>任</span>它留在那裏了。終有一日，<span style='color:red'>村</span>子裏來了一個<span style='color:red'>天</span><span style='color:red'>文</span>學家。他在我家<span style='color:red'>門</span><span style='color:red'>前</span>路過，突<span style='color:red'>然</span>發<span style='color:red'>現</span>了這塊石頭，<span style='color:red'>眼</span>光立即就拉直了。他再沒有離開，就住了下來；以後又來了好些<span style='color:red'>人</span>，都說這是一塊<span style='color:red'>隕</span>石，從<span style='color:red'>天</span>上落下來已經有二<span style='color:red'>三</span>百<span style='color:red'>年</span>了，是一<span style='color:red'>件</span>了不起的東西。不久<span style='color:red'>便</span>來了車，小<span style='color:red'>心</span>翼翼地將它<span style='color:red'>運</span>走了。這使我<span style='color:red'>們</span>都<span style='color:red'>很</span>驚奇，這又怪又醜的石頭，<span style='color:red'>原</span>來是<span style='color:red'>天</span>上的啊！它補過<span style='color:red'>天</span>，在<span style='color:red'>天</span>上發過熱、<span style='color:red'>閃</span>過光，我<span style='color:red'>們</span>的<span style='color:red'>先</span>祖或許仰望過它，它給了他<span style='color:red'>們</span>光明、向往、憧憬；而它落下來了，在污土裏，荒草裏，一躺就//是幾百<span style='color:red'>年</span>了！我<span style='color:red'>感</span>到自己的無知，也<span style='color:red'>感</span>到了醜石的偉大，我<span style='color:red'>甚</span>至<span style='color:red'>怨</span><span style='color:red'>恨</span>它這麼多<span style='color:red'>年</span>竟會默默地<span style='color:red'>忍</span>受着這一切！而我又立即<span style='color:red'>深</span><span style='color:red'>深</span>地<span style='color:red'>感</span>到它那種不屈於誤解、寂寞的生<span style='color:red'>存</span>的偉大。

### 後鼻音

In [165]:
def ng_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("ng"):
            print(ch, py)
            
#ng_in_passage(passages[passage_no-1])

def highlight_ng_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("ng"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_ng_in_passage(passages[passage_no-1])

我<span style='color:red'>常</span><span style='color:red'>常</span>遺憾我家門前那塊醜石，它黑黝黝地臥在那裏，牛似的模<span style='color:red'>樣</span>；誰也不知道是什麼時候留在這裏的，誰也不去理會它。只是麥收時節，門前攤了麥子，奶奶<span style='color:red'>總</span>是說：這塊醜石，多佔地面呀，抽<span style='color:red'>空</span>把它搬走吧。它不<span style='color:red'>像</span>漢白玉那<span style='color:red'>樣</span>的細膩，可以刻字雕花，也不<span style='color:red'>像</span>大<span style='color:red'>青</span>石那<span style='color:red'>樣</span>的<span style='color:red'>光</span>滑，可以<span style='color:red'>供</span>來浣紗捶布。它<span style='color:red'>靜</span><span style='color:red'>靜</span>地臥在那裏，院邊的槐陰沒有庇覆它，花兒也不再在它身邊<span style='color:red'>生</span><span style='color:red'>長</span>。<span style='color:red'>荒</span>草便繁衍出來，枝蔓<span style='color:red'>上</span>下，慢慢地，它<span style='color:red'>竟</span>銹<span style='color:red'>上</span>了綠苔、黑斑。我們這些做孩子的，也討厭起它來，<span style='color:red'>曾</span>合夥要搬走它，但力氣又不足；雖時時咒罵它，嫌棄它，也無可奈何，只好任它留在那裏了。<span style='color:red'>終</span>有一日，村子裏來了一個天文學家。他在我家門前路過，突然發現了這塊石頭，眼<span style='color:red'>光</span>立即就拉直了。他再沒有離開，就住了下來；以後又來了好些人，都說這是一塊隕石，<span style='color:red'>從</span>天<span style='color:red'>上</span>落下來已<span style='color:red'>經</span>有二三百年了，是一件了不起的<span style='color:red'>東</span>西。不久便來了車，小心翼翼地<span style='color:red'>將</span>它運走了。這使我們都很<span style='color:red'>驚</span>奇，這又怪又醜的石頭，原來是天<span style='color:red'>上</span>的啊！它補過天，在天<span style='color:red'>上</span>發過熱、閃過<span style='color:red'>光</span>，我們的先祖或許<span style='color:red'>仰</span><span style='color:red'>望</span>過它，它給了他們<span style='color:red'>光</span><span style='color:red'>明</span>、<span style='color:red'>向</span><span style='color:red'>往</span>、<span style='color:red'>憧</span><span style='color:red'>憬</span>；而它落下來了，在污土裏，<span style='color:red'>荒</span>草裏，一<span style='color:red'>躺</span>就//是幾百年了！我感到自己的無知，也感到了醜石的偉大，我甚至怨恨它這麼多年<span style='color:red'>竟</span>會默默地忍受着這一切！而我又立即深深地感到它那<span style='color:red'>種</span>不屈於誤解、寂寞的<span style='color:red'>生</span>存的偉大。

In [176]:
def highlight_g_ng_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("n"):
            result += formatmd(ch, 'green')
        elif py.endswith("ng"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_g_ng_in_passage(passages[passage_no-1])

我<span style='color:blue'>常</span><span style='color:blue'>常</span>遺<span style='color:green'>憾</span>我家<span style='color:green'>門</span><span style='color:green'>前</span>那塊醜石，它黑黝黝地臥在那裏，牛似的模<span style='color:blue'>樣</span>；誰也不知道是<span style='color:green'>什</span>麼時候留在這裏的，誰也不去理會它。只是麥收時節，<span style='color:green'>門</span><span style='color:green'>前</span><span style='color:green'>攤</span>了麥子，奶奶<span style='color:blue'>總</span>是說：這塊醜石，多<span style='color:green'>佔</span>地<span style='color:green'>面</span>呀，抽<span style='color:blue'>空</span>把它<span style='color:green'>搬</span>走吧。它不<span style='color:blue'>像</span><span style='color:green'>漢</span>白玉那<span style='color:blue'>樣</span>的細膩，可以刻字雕花，也不<span style='color:blue'>像</span>大<span style='color:blue'>青</span>石那<span style='color:blue'>樣</span>的<span style='color:blue'>光</span>滑，可以<span style='color:blue'>供</span>來<span style='color:green'>浣</span>紗捶布。它<span style='color:blue'>靜</span><span style='color:blue'>靜</span>地臥在那裏，<span style='color:green'>院</span><span style='color:green'>邊</span>的槐<span style='color:green'>陰</span>沒有庇覆它，花兒也不再在它<span style='color:green'>身</span><span style='color:green'>邊</span><span style='color:blue'>生</span><span style='color:blue'>長</span>。<span style='color:blue'>荒</span>草<span style='color:green'>便</span><span style='color:green'>繁</span><span style='color:green'>衍</span>出來，枝<span style='color:green'>蔓</span><span style='color:blue'>上</span>下，<span style='color:green'>慢</span><span style='color:green'>慢</span>地，它<span style='color:blue'>竟</span>銹<span style='color:blue'>上</span>了綠苔、黑<span style='color:green'>斑</span>。我<span style='color:green'>們</span>這些做孩子的，也討<span style='color:green'>厭</span>起它來，<span style='color:blue'>曾</span>合夥要<span style='color:green'>搬</span>走它，<span style='color:green'>但</span>力氣又不足；雖時時咒罵它，<span style='color:green'>嫌</span>棄它，也無可奈何，只好<span style='color:green'>任</span>它留在那裏了。<span style='color:blue'>終</span>有一日，<span style='color:green'>村</span>子裏來了一個<span style='color:green'>天</span><span style='color:green'>文</span>學家。他在我家<span style='color:green'>門</span><span style='color:green'>前</span>路過，突<span style='color:green'>然</span>發<span style='color:green'>現</span>了這塊石頭，<span style='color:green'>眼</span><span style='color:blue'>光</span>立即就拉直了。他再沒有離開，就住了下來；以後又來了好些<span style='color:green'>人</span>，都說這是一塊<span style='color:green'>隕</span>石，<span style='color:blue'>從</span><span style='color:green'>天</span><span style='color:blue'>上</span>落下來已<span style='color:blue'>經</span>有二<span style='color:green'>三</span>百<span style='color:green'>年</span>了，是一<span style='color:green'>件</span>了不起的<span style='color:blue'>東</span>西。不久<span style='color:green'>便</span>來了車，小<span style='color:green'>心</span>翼翼地<span style='color:blue'>將</span>它<span style='color:green'>運</span>走了。這使我<span style='color:green'>們</span>都<span style='color:green'>很</span><span style='color:blue'>驚</span>奇，這又怪又醜的石頭，<span style='color:green'>原</span>來是<span style='color:green'>天</span><span style='color:blue'>上</span>的啊！它補過<span style='color:green'>天</span>，在<span style='color:green'>天</span><span style='color:blue'>上</span>發過熱、<span style='color:green'>閃</span>過<span style='color:blue'>光</span>，我<span style='color:green'>們</span>的<span style='color:green'>先</span>祖或許<span style='color:blue'>仰</span><span style='color:blue'>望</span>過它，它給了他<span style='color:green'>們</span><span style='color:blue'>光</span><span style='color:blue'>明</span>、<span style='color:blue'>向</span><span style='color:blue'>往</span>、<span style='color:blue'>憧</span><span style='color:blue'>憬</span>；而它落下來了，在污土裏，<span style='color:blue'>荒</span>草裏，一<span style='color:blue'>躺</span>就//是幾百<span style='color:green'>年</span>了！我<span style='color:green'>感</span>到自己的無知，也<span style='color:green'>感</span>到了醜石的偉大，我<span style='color:green'>甚</span>至<span style='color:green'>怨</span><span style='color:green'>恨</span>它這麼多<span style='color:green'>年</span><span style='color:blue'>竟</span>會默默地<span style='color:green'>忍</span>受着這一切！而我又立即<span style='color:green'>深</span><span style='color:green'>深</span>地<span style='color:green'>感</span>到它那<span style='color:blue'>種</span>不屈於誤解、寂寞的<span style='color:blue'>生</span><span style='color:green'>存</span>的偉大。

### n/l 音分別

In [178]:
def highlight_n_l_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.startswith("n"):
            result += formatmd(ch, 'green')
        elif py.startswith("l"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_n_l_in_passage(passages[passage_no-1])

我常常遺憾我家門前<span style='color:green'>那</span>塊醜石，它黑黝黝地臥在<span style='color:green'>那</span><span style='color:blue'>裏</span>，<span style='color:green'>牛</span>似的模樣；誰也不知道是什麼時候<span style='color:blue'>留</span>在這<span style='color:blue'>裏</span>的，誰也不去<span style='color:blue'>理</span>會它。只是麥收時節，門前攤<span style='color:blue'>了</span>麥子，<span style='color:green'>奶</span><span style='color:green'>奶</span>總是說：這塊醜石，多佔地面呀，抽空把它搬走吧。它不像漢白玉<span style='color:green'>那</span>樣的細<span style='color:green'>膩</span>，可以刻字雕花，也不像大青石<span style='color:green'>那</span>樣的光滑，可以供<span style='color:blue'>來</span>浣紗捶布。它靜靜地臥在<span style='color:green'>那</span><span style='color:blue'>裏</span>，院邊的槐陰沒有庇覆它，花兒也不再在它身邊生長。荒草便繁衍出<span style='color:blue'>來</span>，枝蔓上下，慢慢地，它竟銹上<span style='color:blue'>了</span><span style='color:blue'>綠</span>苔、黑斑。我們這些做孩子的，也討厭起它<span style='color:blue'>來</span>，曾合夥要搬走它，但<span style='color:blue'>力</span>氣又不足；雖時時咒罵它，嫌棄它，也無可<span style='color:green'>奈</span>何，只好任它<span style='color:blue'>留</span>在<span style='color:green'>那</span><span style='color:blue'>裏</span><span style='color:blue'>了</span>。終有一日，村子<span style='color:blue'>裏</span><span style='color:blue'>來</span><span style='color:blue'>了</span>一個天文學家。他在我家門前<span style='color:blue'>路</span>過，突然發現<span style='color:blue'>了</span>這塊石頭，眼光<span style='color:blue'>立</span>即就<span style='color:blue'>拉</span>直<span style='color:blue'>了</span>。他再沒有<span style='color:blue'>離</span>開，就住<span style='color:blue'>了</span>下<span style='color:blue'>來</span>；以後又<span style='color:blue'>來</span><span style='color:blue'>了</span>好些人，都說這是一塊隕石，從天上<span style='color:blue'>落</span>下<span style='color:blue'>來</span>已經有二三百<span style='color:green'>年</span><span style='color:blue'>了</span>，是一件<span style='color:blue'>了</span>不起的東西。不久便<span style='color:blue'>來</span><span style='color:blue'>了</span>車，小心翼翼地將它運走<span style='color:blue'>了</span>。這使我們都很驚奇，這又怪又醜的石頭，原<span style='color:blue'>來</span>是天上的啊！它補過天，在天上發過熱、閃過光，我們的先祖或許仰望過它，它給<span style='color:blue'>了</span>他們光明、向往、憧憬；而它<span style='color:blue'>落</span>下<span style='color:blue'>來</span><span style='color:blue'>了</span>，在污土<span style='color:blue'>裏</span>，荒草<span style='color:blue'>裏</span>，一躺就//是幾百<span style='color:green'>年</span><span style='color:blue'>了</span>！我感到自己的無知，也感到<span style='color:blue'>了</span>醜石的偉大，我甚至怨恨它這麼多<span style='color:green'>年</span>竟會默默地忍受着這一切！而我又<span style='color:blue'>立</span>即深深地感到它<span style='color:green'>那</span>種不屈於誤解、寂寞的生存的偉大。

### 統計

* 平舌音字數:翹舌音字數
* 前鼻音字數:後鼻音字數
* 多音字字數
* 一不變調詞數
* 去聲變調詞數
* 兒化音詞數

In [181]:
def stats_passage(passage):
    zcs_list = []
    zhchsh_list = []
    n_list = []
    ng_list = []
    n_list2 = []
    l_list = []
    
    for py, ch in passage:
        if ((py.startswith("z") or py.startswith("c") or py.startswith("s"))
              and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            zcs_list.append(ch)
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            zhchsh_list.append(ch)
        if py.endswith("n"):
            n_list.append(ch)
        if py.endswith("ng"):
            ng_list.append(ch)
        if py.startswith("n"):
            n_list2.append(ch)
        if py.startswith("l"):
            l_list.append(ch)
        
    print("平舌音字數:翹舌音字數 = %d : %d" % (len(zcs_list), len(zhchsh_list))) 
    print("前鼻音字數:後鼻音字數 = %d : %d" % (len(n_list), len(ng_list))) 
    print("N 音字數: L 音字數 = %d : %d" % (len(n_list2), len(l_list))) 
    
stats_passage(passages[passage_no-1])

平舌音字數:翹舌音字數 = 30 : 74
前鼻音字數:後鼻音字數 = 69 : 45
N 音字數: L 音字數 = 15 : 45
